In [370]:
import numpy as np
from datetime import timedelta, datetime
import calendar
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [371]:
def days360(start_date, end_date, method_eu=False):
    
    start_day = start_date.day
    start_month = start_date.month
    start_year = start_date.year
    end_day = end_date.day
    end_month = end_date.month
    end_year = end_date.year

    if (
        start_day == 31 or
        (
            method_eu is False and
            start_month == 2 and (
                start_day == 29 or (
                    start_day == 28 and
                    calendar.isleap(start_year) is False
                )
            )
        )
    ):
        start_day = 30

    if end_day == 31:
        if method_eu is False and start_day != 30:
            end_day = 1

            if end_month == 12:
                end_year += 1
                end_month = 1
            else:
                end_month += 1
        else:
            end_day = 30

    return (
        end_day + end_month * 30 + end_year * 360 -
        start_day - start_month * 30 - start_year * 360
    )

In [182]:
def risky_price(fecha_valuacion, fecha_vencimiento, nocional, ytm, tasa_cupon, periodo_cupon = 2, risk = [0,1]):
    
    """
    fecha_valuacion: Fecha a la que se hará la valuación.
    
    fecha_vencimiento: Fecha de vencimiento del Bono evaluado.
    
    nocional: Monto al vencimiento del Bono evaluado.
    
    ytm: Tasa de rendimiento del Bono evaluado.
    
    tasa_cupon: Tasa cupón del Bono evaluado.
    
    periodo_cupon: Número de veces al año que el bono paga un cupón.
    
    risk: 
            Primer elemento .- Probabilidad de Incumplimiento de la contraparte.
            Segundo elemento.- Tasa de Recuperación en caso de incumplimiento de la contraparte.
    """
    
    h = risk[0]
    R = risk[1]
    
    cupon = tasa_cupon * nocional / periodo_cupon
    ytm_periodo  = ytm / periodo_cupon
    
    dias_val_venc = days360(fecha_valuacion, fecha_vencimiento)
    periodo_frac = (dias_val_venc % (360/periodo_cupon)) / (360/periodo_cupon)
    periodos_comp = dias_val_venc // (360/periodo_cupon)
    
    desc_int = 1 / (1 + ytm_periodo) ** (np.arange(periodos_comp + 1) + periodo_frac)
    desc_risk = (1-h) ** np.arange(periodos_comp + 2)
    desc_tot = desc_risk[1:] * desc_int
    desc_def = desc_risk[:-1] * desc_int
    
    risky_price = sum(cupon * desc_tot) + (sum(desc_def) * h * R + desc_tot[-1]) * nocional
    
    return risky_price

In [268]:
def prob_default(fecha_valuacion, fecha_vencimiento, nocional, ytm, tasa_cupon, R = 0.25, libor = 0.0067, periodo_cupon = 2, risk = [0,1]):
    
    h0 = 0.1

    constraints = [
    
    {"type": "eq", "fun": lambda h: risky_price(fecha_valuacion,fecha_vencimiento,nocional,ytm,tasa_cupon) - risky_price(fecha_valuacion,fecha_vencimiento,nocional,libor,tasa_cupon,risk = [h,R])}]

    precio_sucio = risky_price(fecha_valuacion,fecha_vencimiento,nocional,ytm,tasa_cupon)
    
    opt  = minimize(lambda h: (risky_price(fecha_valuacion,fecha_vencimiento,nocional,ytm,tasa_cupon) - 
                           risky_price(fecha_valuacion,fecha_vencimiento,nocional,libor,tasa_cupon,risk = [h,R])),
                           h0, constraints = constraints)
    
    return {"PrecioSucio":precio_sucio, "h":opt.x[0]}

In [638]:
params_default = {"fecha_valuacion" : datetime(2020,4,9),
                  "fecha_vencimiento" : datetime(2027,3,13),
                  "nocional" : 100,
                  "periodo_cupon" : 2,
                  "tasa_cupon" : .065,
                  "libor" : 0.0067,
                  "ytm" : .085523,
                  "R" : 0.25}

In [639]:
res = prob_default(**params_default)

In [646]:
def CVA(prob_default, precio_USD, tasa_USD, nocional_fwd, plazo_fwd, pts_fwd, vol, TIIE_periodo, amplitud = 1, n_simul = 10_000):
    
    """
    prob_default: Probabilidad de default estimada de la contraparte.
    
    precio_USD: Precio USD spot.
    
    nocional_fwd: Monto del contrato.
    
    plazo_fwd: Plazo de duración del contrato.
    
    pts_fwd: Recargo al precio USD spot expresado en puntos forward.
    
    vol: Volatilidad del mercado.
    
    TIIE_periodo: TIIE equivalente al periodo del contrato.
    
    amplitud: Amplitud de intervalo para simulaciones del precio USD. Ej. 1 - diaria, 7 - semanal, 30 - mensual
    
    n_simul: Total de simulaciones para precio USD.
    
    """
    
    f = precio_USD + pts_fwd / 10_000
    tasa_MXN = (f/precio_USD * (1 + tasa_USD*plazo/360) - 1) * (360/plazo)
    
    simul = (np.ones(n_simul) * precio_USD)[:,np.newaxis]
    spread_tasas = tasa_MXN - tasa_USD
    periodos = np.arange(start = 0, stop = plazo, step = amplitud)

    for periodo in periodos:

        simul = np.c_[simul,(
                     simul[:,-1] * 
                     np.exp((spread_tasas - 0.5*vol**2)*(amplitud/360) + 
                            (vol*(amplitud/360)**.5) * 
                            np.random.normal(size = n_simul))
                     )]

    mtm = nocional_fwd * (simul[:,:-1] * np.exp(spread_tasas * (plazo - periodos)/360) - f) * np.exp(-tasa_MXN * (plazo - periodos)/360)

    EPE = np.apply_along_axis(lambda x: x[x>=0].mean(), 0, mtm)
    PFE = np.apply_along_axis(lambda x: x[x>=0][x[x>=0]>=np.percentile(x[x>=0],.95)].mean(), 0, mtm)
    
    desc_int = 1 / ((1 + TIIE * 91/360)**((np.arange(len(EPE))+1)/91))
    desc_risk = (1-(1-h)**((periodos+1)/180))-(1-(1-h)**(periodos/180))

    CVA_EPE = sum(EPE * desc_int * desc_risk)
    CVA_PFE = sum(PFE * desc_int * desc_risk)
    
    constraints = [
    
    {"type": "eq", "fun": lambda precio_nuevo: nocional_fwd * (precio_nuevo-f) * np.exp(-tasa_MXN * plazo/360) - CVA_EPE}]

    opt  = minimize(lambda precio_nuevo: nocional_fwd * (precio_nuevo-f) * np.exp(-tasa_MXN * plazo/360) - CVA_EPE,
                               f, constraints = constraints)

    precio_CVA_EPE = opt.x[0]
    
    constraints = [
    
    {"type": "eq", "fun": lambda precio_nuevo: nocional_fwd * (precio_nuevo-f) * np.exp(-tasa_MXN * plazo/360) - CVA_EPE}]

    opt  = minimize(lambda precio_nuevo: nocional_fwd * (precio_nuevo-f) * np.exp(-tasa_MXN * plazo/360) - CVA_PFE,
                               f, constraints = constraints)

    precio_CVA_PFE = opt.x[0]
    
    return ({"CVA":{"EPE":f"${CVA_EPE:0.2f}",
                    "PFE":f"${CVA_PFE:0.2f}",
                    "Precios":
                        {"PrecioFwd":f"${f}",
                         "PrecioRecargado_EPE":f"${precio_CVA_EPE:0.4f}", 
                         "PrecioRecargado_PFE":f"${precio_CVA_PFE:0.4f}"}}})

In [647]:
h = res["h"]

In [648]:
params_contrato = {"prob_default" : h,
                   "precio_USD" : 20.77,
                   "plazo_fwd" : 90,
                   "pts_fwd" : 4118,
                   "tasa_USD" : 0.007841,
                   "vol" : 0.5,
                   "nocional_fwd" : 10_000_000,
                   "TIIE_periodo" : 0.0713,}

In [649]:
CVA(**params_contrato)

{'CVA': {'EPE': '$741475.65',
  'PFE': '$748619.31',
  'Precios': {'PrecioFwd': '$21.1818',
   'PrecioRecargado_EPE': '$21.2576',
   'PrecioRecargado_PFE': '$21.2576'}}}